# Convolutional Neural Networks for Sentence Classification using Ignite

This is a tutorial on using Ignite to train neural network models, setup experiments and validate models.

In this experiment, we'll be replicating [
Convolutional Neural Networks for Sentence Classification by Yoon Kim](https://arxiv.org/abs/1408.5882)! This paper uses CNN for text classification, a task typically reserved for RNNs, Logistic Regression, Naive Bayes.

We want to be able to classify IMDB movie reviews and predict whether the review is positive or negative. IMDB Movie Review dataset comprises of 25000 positive and 25000 negative examples. The dataset comprises of text and label pairs. This is binary classification problem. We'll be using PyTorch to create the model, torchtext to import data and Ignite to train and monitor the models!

Lets get started! 

## Required Dependencies 

In this example we only need torchvision package, assuming that `torch` and `ignite` are already installed. We can install it using `pip`:

`pip install torchtext spacy`

`python -m spacy download en`

## Import Libraries

In [1]:
import random

`torchtext` is a library that provides multiple datasets for NLP tasks, similar to `torchvision`. Below we import the following:
* **data**: A module to setup the data in the form Fields and Labels.
* **datasets**: A module to download NLP datasets.
* **GloVe**: A module to download and use pretrained GloVe embedings.

In [2]:
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe

We import torch, nn and functional modules to create our models! 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

`Ignite` is a High-level library to help with training neural networks in PyTorch. It comes with an `Engine` to setup a training loop, various metrics, handlers and a helpful contrib section! 

Below we import the following:
* **Engine**: Runs a given process_function over each batch of a dataset, emitting events as it goes.
* **Events**: Allows users to attach functions to an `Engine` to fire functions at a specific event. Eg: `EPOCH_COMPLETED`, `ITERATION_STARTED`, etc.
* **Accuracy**: Metric to calculate accuracy over a dataset, for binary, multiclass, multilabel cases. 
* **Loss**: General metric that takes a loss function as a parameter, calculate loss over a dataset.
* **RunningAverage**: General metric to attach to Engine during training. 
* **ModelCheckpoint**: Handler to checkpoint models. 
* **EarlyStopping**: Handler to stop training based on a score function. 
* **ProgressBar**: Handler to create a tqdm progress bar.

In [4]:
from ignite.engine import Engine, Events
from ignite.metrics import Accuracy, Loss, RunningAverage
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.contrib.handlers import ProgressBar

## Processing Data

The code below first sets up `TEXT` and `LABEL` as general data objects. 

* `TEXT` converts any text to lowercase and produces tensors with the batch dimension first. 
* `LABEL` is a data object that will convert any labels to floats.

Next IMDB training and test datasets are downloaded, the training data is split into training and validation datasets. It takes TEXT and LABEL as inputs so that the data is processed as specified. 

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TEXT = data.Field(lower=True, batch_first=True)
LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root='/tmp/imdb/')
train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED))

Now we have three sets of the data - train, validation, test. Let's explore what these data objects are and how to extract data from them.
* `train_data` is **torchtext.data.dataset.Dataset**, this is similar to **torch.utils.data.Dataset**.
* `train_data[0]` is **torchtext.data.example.Example**, a Dataset is comprised of many Examples.
* Let's explore the attributes of an example. We see a few methods, but most importantly we see `label` and `text`.
* `example.text` is the text of that example and `example.label` is the label of the example. 

In [6]:
print('type(train_data) : ', type(train_data))
print('type(train_data[0]) : ',type(train_data[0]))
example = train_data[0]
print('Attributes of Example : ', [attr for attr in dir(example) if '_' not in attr])
print('example.label : ', example.label)
print('example.text[:10] : ', example.text[:10])

type(train_data) :  <class 'torchtext.data.dataset.Dataset'>
type(train_data[0]) :  <class 'torchtext.data.example.Example'>
Attributes of Example :  ['fromCSV', 'fromJSON', 'fromdict', 'fromlist', 'fromtree', 'label', 'text']
example.label :  neg
example.text[:10] :  ['i', 'sat', 'glued', 'to', 'the', 'screen,', 'riveted,', 'yawning,', 'yet', 'keeping']


Now that we have an idea of what are split datasets look like, lets dig further into `TEXT` and `LABEL`. It is important that we build our vocabulary based on the train dataset as validation and test are **unseen** in our experimenting. 

For `TEXT`, let's download the pretrained **GloVE** 100 dimensional word vectors. This means each word is described by 100 floats! If you want to read more about this, here are a few resources.
* [StanfordNLP - GloVe](https://github.com/stanfordnlp/GloVe)
* [DeepLearning.ai Lecture](https://www.coursera.org/lecture/nlp-sequence-models/glove-word-vectors-IxDTG)
* [Stanford CS224N Lecture by Richard Socher](https://www.youtube.com/watch?v=ASn7ExxLZws)

We use `TEXT.build_vocab` to do this, let's explore the `TEXT` object more. 

Let's explore what `TEXT` object is and how to extract data from them. We see `TEXT` has a few attributes, let's explore vocab, since we just used the build_vocab function. 

`TEXT.vocab` has the following attributes:
* `extend` is used to extend the vocabulary
* `freqs` is a dictionary of the frequency of each word
* `itos` is a list of all the words in the vocabulary.
* `stoi` is a dictionary mapping every word to an index.
* `vectors` is a torch.Tensor of the downloaded embeddings

In [7]:
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=100, cache='/tmp/glove/'))
print ('Attributes of TEXT : ', [attr for attr in dir(TEXT) if '_' not in attr])
print ('Attributes of TEXT.vocab : ', [attr for attr in dir(TEXT.vocab) if '_' not in attr])
print ('First 5 values TEXT.vocab.itos : ', TEXT.vocab.itos[0:5]) 
print ('First 5 key, value pairs of TEXT.vocab.stoi : ', {key:value for key,value in list(TEXT.vocab.stoi.items())[0:5]}) 
print ('Shape of TEXT.vocab.vectors.shape : ', TEXT.vocab.vectors.shape)

Attributes of TEXT :  ['dtype', 'dtypes', 'lower', 'numericalize', 'pad', 'postprocessing', 'preprocess', 'preprocessing', 'process', 'sequential', 'tokenize', 'vocab']
Attributes of TEXT.vocab :  ['extend', 'freqs', 'itos', 'stoi', 'vectors']
First 5 values TEXT.vocab.itos :  ['<unk>', '<pad>', 'the', 'a', 'and']
First 5 key, value pairs of TEXT.vocab.stoi :  {'derived)': 134015, 'blum,': 120130, 'cartland': 50785, 'eccentric': 4902, 'designs': 6221}
Shape of TEXT.vocab.vectors.shape :  torch.Size([218957, 100])


Let's do the same with `LABEL`. We see that there are vectors related to `LABEL`, this is expected because `LABEL` provides the definition of a label of data.

In [8]:
LABEL.build_vocab(train_data)
print ('Attributes of LABEL : ', [attr for attr in dir(LABEL) if '_' not in attr])
print ('Attributes of LABEL.vocab : ', [attr for attr in dir(LABEL.vocab) if '_' not in attr])
print ('First 5 values LABEL.vocab.itos : ', LABEL.vocab.itos) 
print ('First 5 key, value pairs of LABEL.vocab.stoi : ', {key:value for key,value in list(LABEL.vocab.stoi.items())})
print ('Shape of LABEL.vocab.vectors : ', LABEL.vocab.vectors)

Attributes of LABEL :  ['dtype', 'dtypes', 'lower', 'numericalize', 'pad', 'postprocessing', 'preprocess', 'preprocessing', 'process', 'sequential', 'tokenize', 'vocab']
Attributes of LABEL.vocab :  ['extend', 'freqs', 'itos', 'stoi', 'vectors']
First 5 values LABEL.vocab.itos :  ['neg', 'pos']
First 5 key, value pairs of LABEL.vocab.stoi :  {'neg': 0, 'pos': 1}
Shape of LABEL.vocab.vectors :  None


Now we must convert our split datasets into iterators, we'll take advantage of **torchtext.data.BucketIterator**! BucketIterator pads every element of a batch to the length of the longest element of the batch.

In [9]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), 
                                                                           batch_size=32,
                                                                           device=device)

Let's actually explore what the output of the iterator is, this way we'll know what the input of the model is, how to compare the label to the output and how to setup are process_functions for Ignite's `Engine`.
* `batch.label[0]` is the label of a single example. We can see that `LABEL.vocab.stoi` was used to map the label that originally text into a float.
* `batch.text[0]` is the text of a single example. Similar to label, `TEXT.vocab.stoi` was used to convert each token of the example's text into indices.

Now let's print the lengths of the sentences of the first 10 batches of `train_iterator`. We see here that all the batches are of different lengths, this means that the bucket iterator is doing exactly as we hoped!

In [10]:
batch = next(iter(train_iterator))
print('batch.label[0] : ', batch.label[0])
print('batch.text[0] : ', batch.text[0][batch.text[0] != 1])

lengths = []
for i, batch in enumerate(train_iterator):
    x = batch.text
    lengths.append(x.shape[1])
    if i == 10:
        break

print ('Lengths of first 10 batches : ', lengths)

batch.label[0] :  tensor(1., device='cuda:0')
batch.text[0] :  tensor([     9,   1107,    205,     10,     24,      4,    454,     12,     49,
          1271,     12,    346,      3,   2793,      6,   3109,     20,    240,
             4,     79,     76,    210,      7,     61,    653,     50,     25,
            26,      3,    469,    502,    264,    313,      3,    457,  21725,
             6,     73,   1090,     92,     89,      6,    209,     67,     12,
             4,    181,      2,     20,  32092,   4102, 173802,    260,     31,
         16607,      4,     21,   2546,  33723,     69,    748,  44230,     90,
            32,    337,   2438,   1509,   4902, 170285,  21837,      3,  47953,
           342,    260,     31,   1673,  10692,     14,      3,    448,   1018,
          2410,      8,      2,     20,      4,     17,      2,    119,     57,
           474,    215,   1779,     14,  12440,   3661,    260,     31,      2,
          2356,      4,   2333,   8324,  24453,  23239,  

## TextCNN Model

Here is the replication of the model, here are the operations of the model:
* **Embedding**: Embeds a batch of text of shape (N, L) to (N, L, D), where N is batch size, L is maximum length of the batch, D is the embedding dimension. 

* **Convolutions**: Runs parallel convolutions across the embedded words with kernel sizes of 3, 4, 5 to mimic trigrams, four-grams, five-grams. This results in outputs of (N, L - k + 1, D) per convolution, where k is the kernel_size. 

* **Activation**: ReLu activation is applied to each convolution operation.

* **Pooling**: Runs parallel maxpooling operations on the activated convolutions with window sizes of L - k + 1, resulting in 1 value per channel i.e. a shape of (N, 1, D) per pooling. 

* **Concat**: The pooling outputs are concatenated and squeezed to result in a shape of (N, 3D). This is a single embedding for a sentence.

* **Dropout**: Dropout is applied to the embedded sentence. 

* **Fully Connected**: The dropout output is passed through a fully connected layer of shape (3D, 1) to give a single output for each example in the batch. sigmoid is applied to the output of this layer.

* **load_embeddings**: This is a method defined for TextCNN to load embeddings based on user input. There are 3 modes - rand which results in randomly initialized weights, static which results in frozen pretrained weights, nonstatic which results in trainable pretrained weights. 


Let's note that this model works for variable text lengths! The idea to embed the words of a sentence, use convolutions, maxpooling and concantenation to embed the sentence as a single vector! This single vector is passed through a fully connected layer with sigmoid to output a single value. This value can be interpreted as the probability a sentence is positive (closer to 1) or negative (closer to 0).

The minimum length of text expected by the model is the size of the smallest kernel size of the model.

In [11]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, kernel_sizes, num_filters, num_classes, d_prob, mode):
        super(TextCNN, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.kernel_sizes = kernel_sizes
        self.num_filters = num_filters
        self.num_classes = num_classes
        self.d_prob = d_prob
        self.mode = mode
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=1)
        self.load_embeddings()
        self.conv = nn.ModuleList([nn.Conv1d(in_channels=embedding_dim,
                                             out_channels=num_filters,
                                             kernel_size=k, stride=1) for k in kernel_sizes])
        self.dropout = nn.Dropout(d_prob)
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)

    def forward(self, x):
        batch_size, sequence_length = x.shape
        x = self.embedding(x).transpose(1, 2)
        x = [F.relu(conv(x)) for conv in self.conv]
        x = [F.max_pool1d(c, c.size(-1)).squeeze(dim=-1) for c in x]
        x = torch.cat(x, dim=1)
        x = self.fc(self.dropout(x))
        return torch.sigmoid(x).squeeze()

    def load_embeddings(self):
        if 'static' in self.mode:
            self.embedding.weight.data.copy_(TEXT.vocab.vectors)
            if 'non' not in self.mode:
                self.embedding.weight.data.requires_grad = False
                print('Loaded pretrained embeddings, weights are not trainable.')
            else:
                self.embedding.weight.data.requires_grad = True
                print('Loaded pretrained embeddings, weights are trainable.')
        elif self.mode == 'rand':
            print('Randomly initialized embeddings are used.')
        else:
            raise ValueError('Unexpected value of mode. Please choose from static, nonstatic, rand.')

## Creating Model, Optimizer and Loss

Below we create an instance of the TextCNN model and load embeddings in **static** mode. The model is placed on a device and then a loss function of Binary Cross Entropy and Adam optimizer are setup. 

In [12]:
vocab_size, embedding_dim = TEXT.vocab.vectors.shape

model = TextCNN(vocab_size=vocab_size,
                embedding_dim=embedding_dim,
                kernel_sizes=[3, 4, 5],
                num_filters=100,
                num_classes=1, 
                d_prob=0.5,
                mode='static')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)
criterion = nn.BCELoss()

Loaded pretrained embeddings, weights are not trainable.


## Training and Evaluating using Ignite

### Trainer Engine - process_function

Ignite's Engine allows user to define a process_function to process a given batch, this is applied to all the batches of the dataset. This is a general class that can be applied to train and validate models! A process_function has two parameters engine and batch. 


Let's walk through what the function of the trainer does:

* Sets model in train mode. 
* Sets the gradients of the optimizer to zero.
* Generate x and y from batch.
* Performs a forward pass to calculate y_pred using model and x.
* Calculates loss using y_pred and y.
* Performs a backward pass using loss to calculate gradients for the model parameters.
* model parameters are optimized using gradients and optimizer.
* Returns scalar loss. 

Below is a single operation during the trainig process. This process_function will be attached to the training engine.

In [13]:
def process_function(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch.text, batch.label
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()

### Evaluator Engine - process_function

Similar to the training process function, we setup a function to evaluate a single batch. Here is what the eval_function does:

* Sets model in eval mode.
* Generates x and y from batch.
* With torch.no_grad(), no gradients are calculated for any succeding steps.
* Performs a forward pass on the model to calculate y_pred based on model and x.
* Returns y_pred and y.

Ignite suggests attaching metrics to evaluators and not trainers because during the training the model parameters are constantly changing and it is best to evaluate model on a stationary model. This information is important as there is a difference in the functions for training and evaluating. Training returns a single scalar loss. Evaluating returns y_pred and y as that output is used to calculate metrics per batch for the entire dataset.

All metrics in Ignite require y_pred and y as outputs of the function attached to the Engine. 

In [14]:
def eval_function(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch.text, batch.label
        y_pred = model(x)
        return y_pred, y

### Instantiating Training and Evaluating Engines

Below we create 3 engines, a trainer, a training evaluator and a validation evaluator. You'll notice that train_evaluator and validation_evaluator use the same function, we'll see later why this was done! 

In [15]:
trainer = Engine(process_function)
train_evaluator = Engine(eval_function)
validation_evaluator = Engine(eval_function)

### Metrics - RunningAverage, Accuracy and Loss

To start, we'll attach a metric of Running Average to track a running average of the scalar loss output for each batch. 

In [16]:
RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')

Now there are two metrics that we want to use for evaluation - accuracy and loss. This is a binary problem, so for Loss we can simply pass the Binary Cross Entropy function as the loss_function. 

For Accuracy, Ignite requires y_pred and y to be comprised of 0's and 1's only. Since our model outputs from a sigmoid layer, values are between 0 and 1. We'll need to write a function that transforms `engine.state.output` which is comprised of y_pred and y. 

Below `thresholded_output_transform` does just that, it rounds y_pred to convert y_pred to 0's and 1's, and then returns rounded y_pred and y. This function is the output_transform function used to transform the `engine.state.output` to achieve `Accuracy`'s desired purpose.

Now, we attach `Loss` and `Accuracy` (with `thresholded_output_transform`) to train_evaluator and validation_evaluator. 

To attach a metric to engine, the following format is used:
* `Metric(output_transform=output_transform, ...).attach(engine, 'metric_name')`


In [17]:
def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(y_pred)
    return y_pred, y

In [18]:
Accuracy(output_transform=thresholded_output_transform).attach(train_evaluator, 'accuracy')
Loss(criterion).attach(train_evaluator, 'bce')

In [19]:
Accuracy(output_transform=thresholded_output_transform).attach(validation_evaluator, 'accuracy')
Loss(criterion).attach(validation_evaluator, 'bce')

### Progress Bar

Next we create an instance of Ignite's progess bar and attach it to the trainer and pass it a key of `engine.state.metrics` to track. In this case, the progress bar will be tracking `engine.state.metrics['loss']`

In [20]:
pbar = ProgressBar(persist=True, bar_format="")
pbar.attach(trainer, ['loss'])

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### EarlyStopping - Tracking Validation Loss

Now we'll setup a Early Stopping handler for this training process. EarlyStopping requires a score_function that allows the user to define whatever criteria to stop trainig. In this case, if the loss of the validation set does not decrease in 5 epochs, the training process will stop early.  

In [21]:
def score_function(engine):
    val_loss = engine.state.metrics['bce']
    return -val_loss

handler = EarlyStopping(patience=5, score_function=score_function, trainer=trainer)
validation_evaluator.add_event_handler(Events.COMPLETED, handler)

### Attaching Custom Functions to Engine at specific Events

Below you'll see ways to define your own custom functions and attaching them to various `Events` of the training process.

The functions below both achieve similar tasks, they print the results of the evaluator run on a dataset. One function does that on the training evaluator and dataset, while the other on the validation. Another difference is how these functions are attached in the trainer engine.

The first method involves using a decorator, the syntax is simple - `@` `trainer.on(Events.EPOCH_COMPLETED)`, means that the decorated function will be attached to the trainer and called at the end of each epoch. 

The second method involves using the add_event_handler method of trainer - `trainer.add_event_handler(Events.EPOCH_COMPLETED, custom_function)`. This achieves the same result as the above. 

In [22]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    train_evaluator.run(train_iterator)
    metrics = train_evaluator.state.metrics
    avg_accuracy = metrics['accuracy']
    avg_bce = metrics['bce']
    pbar.log_message(
        "Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
        .format(engine.state.epoch, avg_accuracy, avg_bce))
    
def log_validation_results(engine):
    validation_evaluator.run(valid_iterator)
    metrics = validation_evaluator.state.metrics
    avg_accuracy = metrics['accuracy']
    avg_bce = metrics['bce']
    pbar.log_message(
        "Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
        .format(engine.state.epoch, avg_accuracy, avg_bce))
    pbar.n = pbar.last_print_n = 0

trainer.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)

### ModelCheckpoint

Lastly, we want to checkpoint this model. It's important to do so, as training processes can be time consuming and if for some reason something goes wrong during training, a model checkpoint can be helpful to restart training from the point of failure.

Below we'll use Ignite's `ModelCheckpoint` handler to checkpoint models at the end of each epoch. 

In [24]:
checkpointer = ModelCheckpoint('/tmp/models', 'textcnn', n_saved=2, create_dir=True, save_as_state_dict=True)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {'textcnn': model})

### Run Engine

Next, we'll run the trainer for 10 epochs and monitor results. Below we can see that progess bar prints the loss per iteration, and prints the results of training and validation as we specified in our custom function. 

In [25]:
trainer.run(train_iterator, max_epochs=20)


Training Results - Epoch: 1  Avg accuracy: 0.76 Avg loss: 0.64
Validation Results - Epoch: 1  Avg accuracy: 0.73 Avg loss: 0.64



Training Results - Epoch: 2  Avg accuracy: 0.79 Avg loss: 0.56
Validation Results - Epoch: 2  Avg accuracy: 0.78 Avg loss: 0.56



Training Results - Epoch: 3  Avg accuracy: 0.80 Avg loss: 0.48
Validation Results - Epoch: 3  Avg accuracy: 0.79 Avg loss: 0.49



Training Results - Epoch: 4  Avg accuracy: 0.81 Avg loss: 0.44
Validation Results - Epoch: 4  Avg accuracy: 0.80 Avg loss: 0.45



Training Results - Epoch: 5  Avg accuracy: 0.82 Avg loss: 0.42
Validation Results - Epoch: 5  Avg accuracy: 0.81 Avg loss: 0.43


...


Training Results - Epoch: 16  Avg accuracy: 0.88 Avg loss: 0.30
Validation Results - Epoch: 16  Avg accuracy: 0.85 Avg loss: 0.34



Training Results - Epoch: 17  Avg accuracy: 0.89 Avg loss: 0.29
Validation Results - Epoch: 17  Avg accuracy: 0.85 Avg loss: 0.34



Training Results - Epoch: 18  Avg accuracy: 0.89 Avg loss: 0.28
Validation Results - Epoch: 18  Avg accuracy: 0.85 Avg loss: 0.34



Training Results - Epoch: 19  Avg accuracy: 0.90 Avg loss: 0.27
Validation Results - Epoch: 19  Avg accuracy: 0.85 Avg loss: 0.33



Training Results - Epoch: 20  Avg accuracy: 0.90 Avg loss: 0.26
Validation Results - Epoch: 20  Avg accuracy: 0.86 Avg loss: 0.33


That's it! We have successfully trained and evaluated a Convolutational Neural Network for Text Classification. 